In [21]:
# Import usual libraries
import numpy as np
import matplotlib.pyplot as plt
import itertools


import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import numpy as np

# 1 - DATA MANIPULATION
import pandas as pd
import numpy as np

# 2 - DATA VISUALISATION
import matplotlib.pyplot as plt
import seaborn as sns

# 3 - STATISTICS
from statsmodels.graphics.gofplots import qqplot

# 4 - MACHINE LEARNING
## 4.1 - Preprocessing

### 4.1.1 - Scalers
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

### 4.1.2 - Encoders
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

### 4.1.3 - Crossvalidation, Training, Model
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor

### 4.1.4 - Evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics  import ConfusionMatrixDisplay

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from math import sqrt

# Make all figures tiny for readability purpose
from matplotlib import rcParams
rcParams['figure.figsize'] = (5,3)

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
import numpy as np

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectPercentile, mutual_info_regression
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd


In [22]:
gdpnow = pd.read_csv('data/gdpnow_daily_df.csv', index_col='Unnamed: 0', parse_dates=True) #date_parser=dateparse)
# Ensure that load_df index is in the same date format
gdpnow.index = pd.to_datetime(gdpnow.index)

In [23]:
Target = 'Advance Estimate From BEA'
Drop = ['GDP Nowcast', 'Quarter being forecasted', 'Advance Estimate From BEA', 'Publication Date of Advance Estimate',
       'Days until advance estimate', 'Forecast Error', 'Data releases']

In [24]:

X = gdpnow.drop(columns=Drop)
y = gdpnow['Advance Estimate From BEA']
y= y[-X.shape[0]:]

In [26]:
X.columns

Index(['PCE', 'PCE Goods', 'PCE Services', 'GPDI', 'Fixed Investment',
       'Business Fixed Investment', 'Equipment',
       'Intellectual Property Products', 'Structures', 'Residential',
       'Government', 'Federal Govt', 'S&L', 'Imports', 'Goods imports',
       'Services imports', 'Exports', 'Goods exports', 'Services exports',
       'Net Exports (current, $Bil 2009)', 'Net Exports (previous, $Bil 2009)',
       'Change in net exports ($Bil 2009)',
       'Previous change in private inventories ($Bil 2009)',
       'Current change in private inventories ($Bil 2009)',
       'Change in inventory investment ($Bil 2009)', 'Final Sales',
       'Final Sales to Domestic Purchasers',
       'Final Sales to Private Domestic Purchasers'],
      dtype='object')

In [27]:
y = y.fillna(method='ffill')
y = pd.Series(y)


In [28]:
from sklearn.model_selection import train_test_split

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)


In [ ]:
## Non Linear PCA
# from sklearn.decomposition import KernelPCA
# from sklearn.pipeline import make_pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler

# # Updated preprocessing pipeline with Kernel PCA
# preproc = make_pipeline(
#     SimpleImputer(strategy='mean'),
#     StandardScaler(),
#     KernelPCA(n_components=None, kernel='rbf')  # Example with RBF kernel do not enter the %
# )

# PCA Adjusted preprocessing pipeline with named steps
# preproc = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler()),
#     ('pca', PCA(n_components=0.95, random_state=42))  # Named step 'pca'
# ])

In [29]:

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile, mutual_info_regression
from sklearn.decomposition import PCA

# Adjusted preprocessing pipeline with named steps
preproc = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler()),

])

preproc_selector = Pipeline([
    ('preprocessing', preproc),  # Include the preprocessing steps with PCA
    ('feature_selection', SelectPercentile(
        mutual_info_regression,
        percentile=90 # Keep 90% of all features
    ))
])

In [ ]:

model = XGBRegressor(random_state=42)

param_distributions = {
    'model__n_estimators': [200],
    'model__learning_rate': [0.1],
    'model__max_depth': [16, 20],
}

pipe = Pipeline([
    ('preprocessor', preproc),
    ('model', model)
])

random_search = RandomizedSearchCV(
    pipe,
    param_distributions=param_distributions,
    n_iter=100,  # Number of parameter settings that are sampled. n_iter trades off runtime vs quality of the solution.
    scoring='neg_mean_absolute_error',  # Assuming MSE is the metric of interest; adjust as needed.
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

random_search.fit(X_train, y_train)

print("Best parameters found: ", random_search.best_params_)
print("Best score found: ", random_search.best_score_)


In [30]:
# genrally higher N_estimators and lower learning rates are more robust
# Best parameters found:  {'model__n_estimators': 150, 'model__max_depth': 6, 'model__learning_rate': 0.1}
# Best score found:  -0.10133019738065827

# Defining the best model
model_best =  XGBRegressor(n_estimators=200, max_depth = 20, learning_rate = 0.1, random_state=42)

# Creating the pipeline with memory caching
pipe_best = make_pipeline(preproc_selector, model_best)

# Assuming X_train and y_train are already defined and contain your training data
score = cross_val_score(pipe_best, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')

# Printing the standard deviation and mean of the cross-validation scores
print(score.std())
print(score.mean())

0.19180801915451046
-1.3517119762362344


In [34]:
pipe_best.fit(X_train,y_train)
y_pred = pd.Series(pipe_best.predict(X_test)).reset_index(drop=True)
y_test = pd.Series(y_test).reset_index(drop=True)

In [35]:

from statsmodels.tsa.stattools import acf

def forecast_accuracy(y_pred: pd.Series, y_true: pd.Series) -> float:

    mape = np.mean(np.abs(y_pred - y_true)/np.abs(y_true))  # Mean Absolute Percentage Error
    me = np.mean(y_pred - y_true)             # ME
    mae = np.mean(np.abs(y_pred - y_true))    # MAE
    mpe = np.mean((y_pred - y_true)/y_true)   # MPE
    rmse = np.mean((y_pred - y_true)**2)**.5  # RMSE
    corr = np.corrcoef(y_pred, y_true)[0,1]   # Correlation between the Actual and the Forecast
    mins = np.amin(np.hstack([y_pred.values.reshape(-1,1), y_true.values.reshape(-1,1)]), axis=1)
    maxs = np.amax(np.hstack([y_pred.values.reshape(-1,1), y_true.values.reshape(-1,1)]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    acf1 = acf(y_pred-y_true, fft=False)[1]                      # Lag 1 Autocorrelation of Error

    forecast = ({
        'mape':mape,
        'me':me,
        'mae': mae,
        'mpe': mpe,
        'rmse':rmse,
        'acf1':acf1,
        'corr':corr,
        'minmax':minmax
    })

    return forecast


In [36]:
forecast_accuracy(y_pred,y_test)

{'mape': 0.4593459016327415,
 'me': -0.15542044026527826,
 'mae': 0.8938565876950898,
 'mpe': 0.12514118918789327,
 'rmse': 3.227314715396498,
 'acf1': -0.0005272892036298338,
 'corr': 0.8996074638210155,
 'minmax': 0.2623156347554233}

In [37]:
model_forecast = pd.DataFrame()

# Assign rounded values of y_pred and y_test to the DataFrame
model_forecast['y_pred'] = y_pred.values
model_forecast['y_test'] = y_test.values

# Display the DataFrame
model_forecast

,y_pred,y_test
0,3.999980,4.0
1,-0.899911,-0.9
2,2.564316,2.6
3,2.186901,2.6
4,2.000376,2.0
...,...,...
503,-0.899843,-0.9
504,4.064069,4.1
505,0.699966,0.7
506,6.499978,6.5


Viewing the PCA

In [ ]:
# Assuming X_train and y_train are your features and target variables respectively
preproc.fit(X_train, y_train)


In [ ]:
# Fit the preprocessing pipeline and transform X_train
X_train_transformed = preproc.fit_transform(X_train)

In [ ]:
import pandas as pd

# Convert the transformed array into a DataFrame
transformed_df = pd.DataFrame(X_train_transformed,
                              columns=[f'Feature_{i}' for i in range(X_train_transformed.shape[1])])

# Display the transformed DataFrame
print(transformed_df)


In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import shap
from xgboost import XGBRegressor

# Fit your model on the transformed data
model = XGBRegressor(n_estimators=200, max_depth=20, learning_rate=0.1, random_state=42)
model.fit(X_train_transformed, y_train)

# Compute SHAP values to identify influential components
explainer = shap.Explainer(model)
shap_values = explainer(X_train_transformed)

# Assuming we're interested in the top 1 component for demonstration
top_components = np.argsort(-np.sum(np.abs(shap_values.values), axis=0))[0:1]

# Extract PCA step from the pipeline
pca = preproc.named_steps['pca']

# Extract PCA loadings for the top influential components
loadings = pca.components_[top_components]

# Assuming X_train is a DataFrame and has column names
feature_names = X_train.columns

# Create a DataFrame for loadings
loading_df = pd.DataFrame(loadings, columns=feature_names, index=[f'Component {i+1}' for i in top_components])

print("PCA Loadings for the top influential components:")
print(loading_df)

In [ ]:
# Assuming loading_df is already created and contains the PCA loadings for Component 1
# Extract loadings for Component 1
component_1_loadings = loading_df.loc['Component 1']

# Sort the loadings by their absolute values in descending order to find the top features
sorted_loadings = component_1_loadings.abs().sort_values(ascending=False)

# Select the top 20 features based on their loadings
top_50_features = sorted_loadings.head(50)

# Convert the series to a list of feature names
top_50_feature_names = top_50_features.index.tolist()

# Optionally, to get the corresponding loading values as well
top_50_feature_values = top_50_features.values.tolist()

print("Top 50 PCA Loadings for Component 1 (Features):", top_50_feature_names)
print("Top 50 PCA Loadings for Component 1 (Values):", top_50_feature_values)